# Recognition of American Sign Language via a Leap Motion

*by Matt Hehn, Ben Campbell, & Chris Cochran, December, 2017*

## Introduction

(What, why, very brief overview of methods and results.)

I'm very interested in games.  The latest developments in the Unity engine amaze me.  So I ....

## Methods

Steps I took.  Resources I used, such as our textbook [Russell and Norvig, 2014]. 

Say in detail what each team member did.

## Results

Show all results.  Intermediate resultw might be shown in above Methods section.  Plots, tables, whatever.

## Conclusions

What I learned.

### References

* [Russell and Norvig, 2014] Stuart Russell and Peter Norvig, [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu/), Publisher. 2014.
* [chuck] me, fldfkafjlak

Your report should contain approximately 1,500 to 5,000 words, in markdown cells.  You can count words by running the following python code in your report directory.

In [2]:
import io
from nbformat import current
import glob
nbfile = glob.glob('*.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Example of Project Report.ipynb is 123
